# Wrangling Exercises

In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer

pd.options.display.max_columns = None

import os
from env import host, username, password
import acquire
import prepare
import wrangle_mall


For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

## Zillow Dataset Exercises

### Acquire and Summarize

Acquire data from the cloud database.

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

In [165]:
def get_zillow():
    
    if os.path.isfile('zillow_data.csv'):
        
        df = pd.read_csv('zillow_data.csv')
        df = df.drop(columns='Unnamed: 0')

        return df

    else:
        
        url = get_connection('zillow')
        query = '''
                SELECT *
                FROM properties_2017
                LEFT JOIN airconditioningtype USING(airconditioningtypeid)
                LEFT JOIN architecturalstyletype USING(architecturalstyletypeid)
                LEFT JOIN heatingorsystemtype USING(heatingorsystemtypeid)
                LEFT JOIN propertylandusetype USING(propertylandusetypeid)
                LEFT JOIN storytype USING(storytypeid)
                LEFT JOIN typeconstructiontype USING(typeconstructiontypeid)
                LEFT JOIN unique_properties USING(parcelid)
                JOIN predictions_2017 USING(id); 
                '''
        df = pd.read_sql(query, url)                
        df.to_csv('zillow_data.csv')

        return df

In [122]:
def get_connection(db, user=username, host=host, password=password):
    '''
    This functions imports my credentials for the Codeup MySQL server to be used to pull data
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [149]:
df = get_zillow()

/var/folders/zk/vf208zwx67vf8nf5b5xtyllw0000gn/T/ipykernel_24404/1415617014.py:5: DtypeWarning: Columns (61,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('zillow_data.csv')


Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [58]:
df.dtypes

id                          int64
parcelid                    int64
typeconstructiontypeid    float64
storytypeid               float64
propertylandusetypeid     float64
                           ...   
storydesc                  object
typeconstructiondesc       object
parcelid.1                  int64
logerror                  float64
transactiondate            object
Length: 68, dtype: object

In [99]:
df['typeconstructiontypeid'].isna().sum()

77448

In [191]:
df = df[df['transactiondate'].str.startswith('2017')]

In [192]:
df['latitude'].isna().sum()

0

In [193]:
df['longitude'].isna().sum()

0

In [75]:
df['id'].nunique()

77613

In [194]:
df.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
id,77613.0,3.880600e+04,2.240509e+04,0.0,19403.0,38806.0,58209.0,77612.0
parcelid,77613.0,1.337946e+07,8.351498e+06,10711745.0,11585740.0,12532258.0,14115107.0,169601949.0
typeconstructiontypeid,166.0,5.987952e+00,1.552301e-01,4.0,6.0,6.0,6.0,6.0
storytypeid,45.0,7.000000e+00,0.000000e+00,7.0,7.0,7.0,7.0,7.0
propertylandusetypeid,77613.0,2.599353e+02,1.696527e+01,31.0,261.0,261.0,261.0,275.0


In [196]:
df.dtypes

id                          int64
parcelid                    int64
typeconstructiontypeid    float64
storytypeid               float64
propertylandusetypeid     float64
                           ...   
storydesc                  object
typeconstructiondesc       object
parcelid.1                  int64
logerror                  float64
transactiondate            object
Length: 68, dtype: object

In [197]:
df.shape

(77613, 68)

In [198]:
for col in df.columns:
    print ("---- %s ---" % col)
    print (df[col].value_counts())

---- id ---
0        1
51748    1
51746    1
51745    1
51744    1
        ..
25871    1
25870    1
25869    1
25868    1
77612    1
Name: id, Length: 77613, dtype: int64
---- parcelid ---
10754147    1
12342442    1
12342129    1
12341748    1
12341725    1
           ..
14005894    1
14005825    1
14005687    1
14005664    1
11213162    1
Name: parcelid, Length: 77613, dtype: int64
---- typeconstructiontypeid ---
6.0    165
4.0      1
Name: typeconstructiontypeid, dtype: int64
---- storytypeid ---
7.0    45
Name: storytypeid, dtype: int64
---- propertylandusetypeid ---
261.0    56079
266.0    12491
246.0     2852
263.0     1681
269.0     1609
247.0     1021
248.0      980
265.0      288
31.0       272
47.0       144
260.0      103
267.0       48
275.0       35
264.0        9
270.0        1
Name: propertylandusetypeid, dtype: int64
---- heatingorsystemtypeid ---
2.0     30567
7.0     16593
6.0       662
24.0      511
20.0       84
13.0       38
18.0       14
12.0        2
1.0         

In [199]:
df.isna().sum()

id                            0
parcelid                      0
typeconstructiontypeid    77447
storytypeid               77568
propertylandusetypeid         0
                          ...  
storydesc                 77568
typeconstructiondesc      77447
parcelid.1                    0
logerror                      0
transactiondate               0
Length: 68, dtype: int64

In [200]:
df['typeconstructiontypeid'].value_counts()

6.0    165
4.0      1
Name: typeconstructiontypeid, dtype: int64

Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [69]:
df.dropna(axis='columns', how='any')

,id,parcelid,propertylandusetypeid,bathroomcnt,bedroomcnt,fips,latitude,longitude,rawcensustractandblock,regionidcounty,assessmentyear,propertylandusedesc,parcelid.1,logerror,transactiondate
0,0,10754147,269.0,0.0,0.0,6037.0,34144442.0,-118654084.0,6.037800e+07,3101.0,2016.0,Planned Unit Development,14297519,0.025595,2017-01-01
1,1,10759547,261.0,0.0,0.0,6037.0,34140430.0,-118625364.0,6.037800e+07,3101.0,2015.0,Single Family Residential,17052889,0.055619,2017-01-01
2,2,10843547,47.0,0.0,0.0,6037.0,33989359.0,-118394633.0,6.037703e+07,3101.0,2016.0,Store/Office (Mixed Use),14186244,0.005383,2017-01-01
3,3,10859147,47.0,0.0,0.0,6037.0,34148863.0,-118437206.0,6.037141e+07,3101.0,2016.0,Store/Office (Mixed Use),12177905,-0.103410,2017-01-01
4,4,10879947,31.0,0.0,0.0,6037.0,34194168.0,-118385816.0,6.037123e+07,3101.0,2016.0,Commercial/Office/Residential Mixed Used,10887214,0.006940,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,77609,11212539,261.0,3.0,4.0,6037.0,34572256.0,-118024872.0,6.037911e+07,3101.0,2016.0,Single Family Residential,11000655,0.020615,2017-09-20
77610,77610,11212639,261.0,3.0,4.0,6037.0,34571498.0,-118039412.0,6.037911e+07,3101.0,2016.0,Single Family Residential,17239384,0.013209,2017-09-21
77611,77611,11212962,261.0,2.0,3.0,6037.0,34575327.0,-118054267.0,6.037911e+07,3101.0,2016.0,Single Family Residential,12773139,0.037129,2017-09-21
77612,77612,11213162,266.0,3.0,3.0,6037.0,34572200.0,-118053000.0,6.037911e+07,3101.0,2016.0,Condominium,12826780,0.007204,2017-09-25


In [125]:
df.isna().sum() / len(df) < .9

id                         True
parcelid                   True
typeconstructiontypeid    False
storytypeid               False
propertylandusetypeid      True
                          ...  
storydesc                 False
typeconstructiondesc      False
parcelid.1                 True
logerror                   True
transactiondate            True
Length: 68, dtype: bool

In [124]:
df[col].isna().sum() / len(df[col]) < .9

True

In [100]:
df['typeconstructiontypeid']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
77609   NaN
77610   NaN
77611   NaN
77612   NaN
77613   NaN
Name: typeconstructiontypeid, Length: 77614, dtype: float64

In [133]:
if df[col].isna().sum() / len(df[col]) < .9:
            
    df = df.dropna(axis=1, thresh=20000)

In [120]:
for col in df.columns:
    
    if df[col].isna().sum() / len(df[col]) < .9:
            
        df = df.dropna(axis=1)
        
    else:
        
        return df

SyntaxError: 'return' outside function (1344634646.py, line 9)

In [111]:
df

,id,parcelid,propertylandusetypeid,bathroomcnt,bedroomcnt,fips,latitude,longitude,rawcensustractandblock,regionidcounty,assessmentyear,propertylandusedesc,parcelid.1,logerror,transactiondate
0,0,10754147,269.0,0.0,0.0,6037.0,34144442.0,-118654084.0,6.037800e+07,3101.0,2016.0,Planned Unit Development,14297519,0.025595,2017-01-01
1,1,10759547,261.0,0.0,0.0,6037.0,34140430.0,-118625364.0,6.037800e+07,3101.0,2015.0,Single Family Residential,17052889,0.055619,2017-01-01
2,2,10843547,47.0,0.0,0.0,6037.0,33989359.0,-118394633.0,6.037703e+07,3101.0,2016.0,Store/Office (Mixed Use),14186244,0.005383,2017-01-01
3,3,10859147,47.0,0.0,0.0,6037.0,34148863.0,-118437206.0,6.037141e+07,3101.0,2016.0,Store/Office (Mixed Use),12177905,-0.103410,2017-01-01
4,4,10879947,31.0,0.0,0.0,6037.0,34194168.0,-118385816.0,6.037123e+07,3101.0,2016.0,Commercial/Office/Residential Mixed Used,10887214,0.006940,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,77609,11212539,261.0,3.0,4.0,6037.0,34572256.0,-118024872.0,6.037911e+07,3101.0,2016.0,Single Family Residential,11000655,0.020615,2017-09-20
77610,77610,11212639,261.0,3.0,4.0,6037.0,34571498.0,-118039412.0,6.037911e+07,3101.0,2016.0,Single Family Residential,17239384,0.013209,2017-09-21
77611,77611,11212962,261.0,2.0,3.0,6037.0,34575327.0,-118054267.0,6.037911e+07,3101.0,2016.0,Single Family Residential,12773139,0.037129,2017-09-21
77612,77612,11213162,266.0,3.0,3.0,6037.0,34572200.0,-118053000.0,6.037911e+07,3101.0,2016.0,Condominium,12826780,0.007204,2017-09-25


In [86]:
df.columns

Index(['id', 'parcelid', 'propertylandusetypeid', 'bathroomcnt', 'bedroomcnt',
       'fips', 'latitude', 'longitude', 'rawcensustractandblock',
       'regionidcounty', 'assessmentyear', 'propertylandusedesc', 'parcelid.1',
       'logerror', 'transactiondate'],
      dtype='object')

In [ ]:
def prep_zillow(df):
    
    for col in df:
        
        if df.isna().sum() / len(df) < .6:
            
            df.dropna(axis=1)
            
    prop_required_col = 
    
    prop_required_row = 

In [34]:
df.dropna(axis=0, thresh=.75)

,id,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,architecturalstyletypeid,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,parcelid.1,logerror,transactiondate
0,0,10754147,NaN,NaN,269.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,34144442.0,-118654084.0,85768.0,NaN,NaN,NaN,NaN,NaN,010D,NaN,6.037800e+07,37688.0,3101.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Planned Unit Development,NaN,NaN,14297519,0.025595,2017-01-01
1,1,10759547,NaN,NaN,261.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,34140430.0,-118625364.0,4083.0,NaN,NaN,NaN,NaN,NaN,0109,LCA11*,6.037800e+07,37688.0,3101.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,17052889,0.055619,2017-01-01
2,2,10843547,NaN,NaN,47.0,NaN,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,NaN,73026.0,NaN,NaN,73026.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,33989359.0,-118394633.0,63085.0,NaN,NaN,NaN,NaN,NaN,1200,LAC2,6.037703e+07,51617.0,3101.0,NaN,96095.0,0.0,NaN,2.0,NaN,NaN,1959.0,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN,NaN,NaN,NaN,Store/Office (Mixed Use),NaN,NaN,14186244,0.005383,2017-01-01
3,3,10859147,NaN,NaN,47.0,NaN,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,NaN,5068.0,NaN,NaN,5068.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,34148863.0,-118437206.0,7521.0,NaN,NaN,NaN,NaN,NaN,1200,LAC2,6.037141e+07,12447.0,3101.0,27080.0,96424.0,0.0,NaN,NaN,NaN,NaN,1948.0,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN,NaN,NaN,NaN,Store/Office (Mixed Use),NaN,NaN,12177905,-0.103410,2017-01-01
4,4,10879947,NaN,NaN,31.0,NaN,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,NaN,1776.0,NaN,NaN,1776.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,34194168.0,-118385816.0,8512.0,NaN,NaN,NaN,NaN,NaN,1210,LAM1,6.037123e+07,12447.0,3101.0,46795.0,96450.0,0.0,NaN,1.0,NaN,NaN,1947.0,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN,NaN,NaN,NaN,Commercial/Office/Residential Mixed Used,NaN,NaN,10887214,0.006940,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,77609,11212539,NaN,NaN,261.0,2.0,NaN,1.0,NaN,3.0,4.0,NaN,8.0,3.0,NaN,NaN,1921.0,1921.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34572256.0,-118024872.0,6763.0,NaN,NaN,NaN,NaN,NaN,0100,PDA21*,6.037911e+07,40227.0,3101.0,NaN,97330.0,0.0,NaN,1.0,NaN,NaN,1990.0,NaN,NaN,129566.0,162019.0,2016.0,32453.0,2860.33,NaN,NaN,6.037911e+13,Central,NaN,Central,Single Family Residential,NaN,NaN,11000655,0.020615,2017-09-20
77610,77610,11212639,NaN,NaN,261.0,2.0,NaN,1.0,NaN,3.0,4.0,NaN,8.0,3.0,NaN,NaN,1780.0,1780.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34571498.0,-118039412.0,6822.0,NaN,NaN,NaN,NaN,NaN,0100,PDA21*,6.037911e+07,40227.0,3101.0,NaN,97330.0,0.0,NaN

In [18]:
df.head()

,id,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,architecturalstyletypeid,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,parcelid.1,logerror,transactiondate,percent_missing
0,0,10754147,NaN,NaN,269.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,34144442.0,-118654084.0,85768.0,NaN,NaN,NaN,NaN,NaN,010D,NaN,6.037800e+07,37688.0,3101.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Planned Unit Development,NaN,NaN,14297519,0.025595,2017-01-01,NaN
1,1,10759547,NaN,NaN,261.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,34140430.0,-118625364.0,4083.0,NaN,NaN,NaN,NaN,NaN,0109,LCA11*,6.037800e+07,37688.0,3101.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,17052889,0.055619,2017-01-01,NaN
2,2,10843547,NaN,NaN,47.0,NaN,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,NaN,73026.0,NaN,NaN,73026.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,33989359.0,-118394633.0,63085.0,NaN,NaN,NaN,NaN,NaN,1200,LAC2,6.037703e+07,51617.0,3101.0,NaN,96095.0,0.0,NaN,2.0,NaN,NaN,1959.0,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN,NaN,NaN,NaN,Store/Office (Mixed Use),NaN,NaN,14186244,0.005383,2017-01-01,NaN
3,3,10859147,NaN,NaN,47.0,NaN,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,NaN,5068.0,NaN,NaN,5068.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,34148863.0,-118437206.0,7521.0,NaN,NaN,NaN,NaN,NaN,1200,LAC2,6.037141e+07,12447.0,3101.0,27080.0,96424.0,0.0,NaN,NaN,NaN,NaN,1948.0,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN,NaN,NaN,NaN,Store/Office (Mixed Use),NaN,NaN,12177905,-0.103410,2017-01-01,NaN
4,4,10879947,NaN,NaN,31.0,NaN,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,NaN,1776.0,NaN,NaN,1776.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,34194168.0,-118385816.0,8512.0,NaN,NaN,NaN,NaN,NaN,1210,LAM1,6.037123e+07,12447.0,3101.0,46795.0,96450.0,0.0,NaN,1.0,NaN,NaN,1947.0,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN,NaN,NaN,NaN,Commercial/Office/Residential Mixed Used,NaN,NaN,10887214,0.006940,2017-01-01,NaN


In [14]:
def missing_values(df):
    
    null_df = df.isna().sum().reset_index()

    null_df['percent_missing'] = (null_df[0] / len(df)) * 100
    
    null_df = null_df.rename(columns= {'index':'attributes',
                                        0: 'num_missing'})                
    return null_df

In [15]:
missing_values(df)

,attributes,num_missing,percent_missing
0,id,0,0.000000
1,parcelid,0,0.000000
2,typeconstructiontypeid,77448,99.786121
3,storytypeid,77569,99.942021
4,propertylandusetypeid,0,0.000000
...,...,...,...
64,typeconstructiondesc,77448,99.786121
65,parcelid.1,0,0.000000
66,logerror,0,0.000000
67,transactiondate,0,0.000000


### Prepare

Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.


In [218]:
df = df[df['propertylandusetypeid'] == 261]

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

- The input:
    - A dataframe
    - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
    - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
- hint:
    - Look up the dropna documentation.
    - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.

In [19]:
df.isna().sum()

id                            0
parcelid                      0
typeconstructiontypeid    77448
storytypeid               77569
propertylandusetypeid         0
                          ...  
typeconstructiondesc      77448
parcelid.1                    0
logerror                      0
transactiondate               0
percent_missing           77614
Length: 69, dtype: int64

In [ ]:
def prep_zillow(df, col_p=.6, row_p=.75):
    
    null_df = df.isna().sum().reset_index()

    null_df['percent_missing'] = (df[0] / len(df)) * 100
        
    

Encapsulate your work inside of functions in a wrangle_zillow.py module.



### Mall Customers Exercises

 Acquire data from the customers table in the mall_customers database.


In [136]:
mall = acquire.get_mall_customers()

Summarize the data (include distributions and descriptive statistics).


In [3]:
mall.head()

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [4]:
mall.describe().T

,count,mean,std,min,25%,50%,75%,max
customer_id,200.0,100.50,57.879185,1.0,50.75,100.5,150.25,200.0
age,200.0,38.85,13.969007,18.0,28.75,36.0,49.00,70.0
annual_income,200.0,60.56,26.264721,15.0,41.50,61.5,78.00,137.0
spending_score,200.0,50.20,25.823522,1.0,34.75,50.0,73.00,99.0


In [5]:
mall.dtypes

customer_id        int64
gender            object
age                int64
annual_income      int64
spending_score     int64
dtype: object

In [6]:
mall.isna().sum()

customer_id       0
gender            0
age               0
annual_income     0
spending_score    0
dtype: int64

Detect outliers using IQR.


In [7]:
my_list = ['age', 'annual_income', 'spending_score']

prepare.remove_outliers(mall, 1.5, my_list)

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
193,194,Female,38,113,91
194,195,Female,47,120,16
195,196,Female,35,120,79
196,197,Female,45,126,28


Split data into train, validate, and test.


In [8]:
train, validate, test = prepare.subset_df(mall)

(120, 5) (40, 5) (40, 5)


Encode categorical columns using a one hot encoder (pd.get_dummies).


In [ ]:
def wrangle_mall_df(outlier_k=1.5):
    
    df = get_mall_customers()
    df = df.dropna()
    
    cont_list = ['age', 'annual_income', 'spending_score']
    cat_list = ['gender']

    df = remove_outliers(df, 1.5, my_list) 
    df = pd.get_dummies(df, cat_list)
    
    train, validate, test = subset_df(df)
    
    return train, validate, test

In [2]:
train, validate, test = wrangle_mall.wrangle_mall_df()

(118, 6) (40, 6) (40, 6)


Handles missing values.


In [4]:
train.dropna()

,customer_id,age,annual_income,spending_score,gender_Female,gender_Male
22,23,46,25,5,1,0
148,149,34,78,22,1,0
146,147,48,77,36,0,1
33,34,18,33,92,0,1
11,12,35,19,99,1,0
...,...,...,...,...,...,...
106,107,66,63,50,1,0
14,15,37,20,13,0,1
92,93,48,60,49,0,1
179,180,35,93,90,0,1


In [10]:
train.isna().sum()

customer_id       0
age               0
annual_income     0
spending_score    0
gender_Female     0
gender_Male       0
dtype: int64

Scaling


In [11]:
to_scale = ['age', 'annual_income', 'spending_score']

train_scaled, validate_scaled, test_scaled = prepare.scale_data(train, validate, test, MinMaxScaler(), to_scale)

In [14]:
train_scaled.head()

,customer_id,age,annual_income,spending_score,gender_Female,gender_Male
2,3,0.038462,0.008197,0.051020,1,0
77,78,0.423077,0.319672,0.479592,0,1
46,47,0.615385,0.204918,0.551020,1,0
187,188,0.192308,0.704918,0.683673,0,1
191,192,0.269231,0.721311,0.693878,1,0


Encapsulate your work in a wrangle_mall.py python module.



# Class Review takeaways

In [166]:
df = get_zillow()

/var/folders/zk/vf208zwx67vf8nf5b5xtyllw0000gn/T/ipykernel_24404/1415617014.py:5: DtypeWarning: Columns (61,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('zillow_data.csv')


In [184]:
def null_counter(df):
    
    new_cols = ['name', 'num_rows_missing', 'percent_rows_missing'] 
    new_df = pd.DataFrame(columns=new_cols)
    
    for columns in df.columns:
        
        num_missing = df[col].isna().sum()
        pct_missing = num_missing / df.shape[0]
        
        add_df = pd.DataFrame([{'name': col,
                                'num_rows_missing': num_missing,
                                'percent_rows_missing': pct_missing}])
        new_df = pd.concat([new_df, add_df], axis=0)
        
    new_df.set_index('name', inplace=True)
    
    return new_df

In [185]:
new_df = null_counter(df)

In [186]:
new_df

,num_rows_missing,percent_rows_missing
name,,
transactiondate,0,0.0
transactiondate,0,0.0
transactiondate,0,0.0
transactiondate,0,0.0
transactiondate,0,0.0
...,...,...
transactiondate,0,0.0
transactiondate,0,0.0
transactiondate,0,0.0


In [193]:
def drop(df, thresh1, thresh2):
    
    loop = list(range(0, len(df.axes[1])))
    adf = []
    
    for i in loop:
        
        x = df.iloc[:,i].isna().sum()
        adf.append(x)
        
    fjf = []
    
    for i in adf:
        
        x = i / len(df.axes[0])
        fjf.append(x)
        
    indices = np.array(fjf)
    indices2 = np.where(indices > thresh1)[0]
    
    df = df.drop(df.columns[indices2], axis = 1)
    
    ddd = []
    
    x = df.isna().sum(axis=1) 
    
    for i in x:
        
        you = i /len(x)
        ddd.append(you)
        
    indices3 = np.where(ddd)
    indices4 = np.where(indices > thresh2)[0]
        
    drop = df.drop(indices4)
    
    return df

In [181]:
df1 = drop(df, .6, .5)

df1.isna().sum()

Unnamed: 0                          0
id                                  0
parcelid                            0
propertylandusetypeid               0
heatingorsystemtypeid           29139
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           27292
calculatedbathnbr                3223
calculatedfinishedsquarefeet     1112
finishedsquarefeet12             6650
fips                                0
fullbathcnt                      3223
latitude                            0
longitude                           0
lotsizesquarefeet                7079
propertycountylandusecode           1
propertyzoningdesc              26252
rawcensustractandblock              0
regionidcity                     1526
regionidcounty                      0
regionidzip                       275
roomcnt                             1
unitcnt                         26324
yearbuilt                        1170
structuretaxvaluedollarcnt       1265
taxvaluedoll

In [ ]:
cols = mall.columns.to_list()

for col in cols:
    
    plt.hist(mall)
    plt.show()

In [191]:
mall['age'].quantile([.25, .75])

0.25    28.75
0.75    49.00
Name: age, dtype: float64

In [192]:
q1, q3 = np.percentile(mall['age'], [25, 75])

q1, q3

(28.75, 49.0)